In [0]:
# make sure these files are there
# newstories.pickle



# these are created later
# words_vocab.pkl
# wordlist.pickle   # wordlist had only some 10000
# wordlist_full.pickle # this will have the full thing

In [1]:
from __future__ import print_function
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, Input, Flatten, Bidirectional
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.metrics import categorical_accuracy
import numpy as np
import random
import sys
import os
import time
import codecs
import collections
from six.moves import cPickle
import pandas as pd
import json
import pickle

Using TensorFlow backend.


In [0]:
#import spacy, and english model
import spacy
nlp = spacy.load('en')

In [0]:
seq_length = 10 # sequence length, we can change this if we want. this means our input sequences have to contain 30 words
sequences_step = 1 #step to create sequences. each next sequence will be one word ahead of the previous (if step=1)

#full document -  aman hi how are you doing whats up
# 1st seq - aman hi how are you
# 2nd seq - hi how are you doing
# 3rd seq - how are you doing whats
# 4rd seq - are you doing whats up

In [4]:
# new stories contains the first 10 from gutenberg nltk. we hoped for better english sentences
# list of sentences - 52000
with open("oldstories.pickle", "rb") as f:
    sentences = pickle.load(f)

print(sentences[2]) #print any sentence you want
print(len(sentences))


That it was far above the
earth was no expression for it; to the two men in it, it seemed
to be far above the stars.
42944


In [0]:
def create_wordlist(doc):
    wl = []
    for word in doc:
        if "\n" in word.text:
            continue
        if "\r" in word.text:
            continue
        if word.text not in ("\n","\n\n", "\n\n\n",'\u2009','\xa0'):
            wl.append(word.text.lower())
        
    return wl

In [32]:
# this takes a lot of time
wordlist = []
for sent in sentences:
    doc = nlp(sent) # nlp is the english spacy model
    wl = create_wordlist(doc)
    wordlist = wordlist + wl

print(len(wordlist)) # entire set of words


# dump the entire wordlist, because it took loads of time!
# run this cell only for the first time
with open("wordlist_old_fullsentences.pickle", "wb") as f:
    pickle.dump(wordlist, f)

1054669


In [0]:
vocab_file = os.path.join("words_vocab.pkl") #essentially dont need this, but there for the sake of brevity

In [7]:
with open("wordlist_old_fullsentences.pickle", "rb") as f:
    wordlist = pickle.load(f)

# with open("wordlist_full.pickle", "rb") as f:
#     wordlist = pickle.load(f)   #calling the variable wordlist only. you can choose to load wordlist_full or wordlist

# count the number of words
word_counts = collections.Counter(wordlist)

# Mapping from index to word : that's the vocabulary
vocabulary_inv = [x[0] for x in word_counts.most_common()]
vocabulary_inv = list(sorted(vocabulary_inv))

# Mapping from word to index
vocab = {x: i for i, x in enumerate(vocabulary_inv)}
words = [x[0] for x in word_counts.most_common()]

#size of the vocabulary
vocab_size = len(words)
print("number of unique words: ", vocab_size)

#save the words and vocabulary
# words - list of words sorted acc to frequency (decreasing) 
# vocab - dictionary {word:index}
# vocab inv - list of alphabetically sorted words

with open(os.path.join(vocab_file), 'wb') as f:
    cPickle.dump((words, vocab, vocabulary_inv), f)

# words_vocab is oldstories and only 10000 sentences

number of unique words:  36565


In [8]:
#create sequences
sequences = []
next_words = []
for i in range(0, len(wordlist) - seq_length, sequences_step):
    sequences.append(wordlist[i: i + seq_length])   #this contains a sequence, seq_length = 30
    next_words.append(wordlist[i + seq_length])     # this is word that occurs right after the sequence

    # seq = [aman hi how are you, ]
    # next =[doing, ]

print('nb sequences:', len(sequences))

nb sequences: 1054659


In [0]:
# from random import sample
# size = 10000 #this can be changed
# rand_sequences = sample(sequences, size)
# sequences = rand_sequences

# print(len(sequences))

In [0]:
size = 50000
# this creates a matrix. logic is explained.
X = np.zeros((size, seq_length, vocab_size), dtype=np.bool)
y = np.zeros((size, vocab_size), dtype=np.bool)
for i, sentence in enumerate(sequences[:size]):
    for t, word in enumerate(sentence):
        X[i, t, vocab[word]] = 1
    y[i, vocab[next_words[i]]] = 1

In [0]:
def bidirectional_lstm_model(seq_length, vocab_size):
    print('Build LSTM model.')
    model = Sequential()
    model.add(Bidirectional(LSTM(rnn_size, activation="relu"),input_shape=(seq_length, vocab_size)))
    model.add(Dropout(0.6))
    model.add(Dense(vocab_size))
    model.add(Activation('softmax'))
    
    optimizer = Adam(lr=learning_rate)
    callbacks=[EarlyStopping(patience=2, monitor='val_loss')]
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=[categorical_accuracy])
    return model

In [0]:
rnn_size = 256 # size of RNN
batch_size = 32 # minibatch size
seq_length = 10 # sequence length
num_epochs = 10 # number of epochs
learning_rate = 0.001 #learning rate
sequences_step = 1 #step to create sequences

In [12]:
md = bidirectional_lstm_model(seq_length, vocab_size)
md.summary()

Build LSTM model.




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 512)               75411456  
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 36565)             18757845  
_________________________________________________________________
activation_1 (Activation)    (None, 36565)             0         
Total params: 94,169,301
Trainable params: 94,169,301
Non-trainable params: 0
_________________________________________________________________


In [13]:
#fit the model
callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
           ModelCheckpoint(filepath='my_model_gen_sentences_lstm.{epoch:02d}-{val_loss:.2f}.hdf5',\
                           monitor='val_loss', verbose=0, mode='auto', period=2)]
history = md.fit(X, y,
                 batch_size=batch_size,
                 shuffle=True,
                 epochs=num_epochs,
                 callbacks=callbacks,
                 validation_split=0.01)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 49500 samples, validate on 500 samples
Epoch 1/10





49500/49500 [==============================] - 215s 4ms/step - loss: 6.5910 - categorical_accuracy: 0.0617 - val_loss: 5.9868 - val_categorical_accuracy: 0.1300
Epoch 2/10
49500/49500 [==============================] - 206s 4ms/step - loss: 5.9450 - categorical_accuracy: 0.0998 - val_loss: 5.7195 - val_categorical_accuracy: 0.1760
Epoch 3/10
49500/49500 [==============================] - 208s 4ms/step - loss: 5.6133 - categorical_accuracy: 0.1217 - val_loss: 5.5911 - val_categorical_accuracy: 0.1800
Epoch 4/10
49500/49500 [==============================] - 207s 4ms/step - loss: 5.3240 - categorical_accuracy: 0.1469 - val_loss: 5.4981 - val_categorical_accuracy: 0.1900
Epoch 5/10
49500/49500 [==============================] - 207s 4ms/step - loss: 5.0380 - categorical_accuracy: 0.1708 - val_loss: 5.4560 - val_categorical_accurac

In [0]:
#save the model
md.save('my_model_gen_sentences_lstm.final.hdf5')

In [15]:
#load vocabulary
print("loading vocabulary...")
vocab_file = os.path.join("words_vocab.pkl")

with open(os.path.join('words_vocab.pkl'), 'rb') as f:
        words, vocab, vocabulary_inv = cPickle.load(f)

vocab_size = len(words)

loading vocabulary...


In [16]:
from keras.models import load_model
# load the model
print("loading model...")
model = load_model('my_model_gen_sentences_lstm.final.hdf5')

loading model...


In [0]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [41]:
#initiate sentences
seed_sentences = "through experience of this great event"
generated = ''
sentence = []
for i in range (seq_length):
    sentence.append("a")

seed = seed_sentences.split()

for i in range(len(seed)):
    sentence[seq_length-i-1]=seed[len(seed)-i-1]

generated += ' '.join(sentence)
print('Generating text with the following seed: "' + ' '.join(sentence) + '"')

print ()

Generating text with the following seed: "a a a a through experience of this great event"



In [42]:
words_number = 10
#generate the text
for i in range(words_number):
    #create the vector
    x = np.zeros((1, seq_length, vocab_size))
    for t, word in enumerate(sentence):
        x[0, t, vocab[word]] = 1.
    #print(x.shape)

    #calculate next word
    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, 1.0)
    next_word = vocabulary_inv[next_index]

    #add the next word to the text
    generated += " " + next_word
    # shift the sentence by one, and and the next word at its end
    sentence = sentence[1:] + [next_word]

print(generated)

a a a a through experience of this great event god said merely hour towards two is men were which
